In [ ]:
! pip install pytubefix opencv-python

scarica una lista di video in base ad una query di ricerca

In [ ]:
import os
import cv2
from pytubefix import YouTube, Search

def cerca_nuovi_video(query, num_pagine, file_storico):
    storico_url = set()
    nuovi_url = []

    # Leggi file con gli URL già scaricati
    if os.path.exists(file_storico):
        with open(file_storico, "r") as f:
            storico_url = set(line.strip() for line in f if line.strip())

    # Cerca i video
    s = Search(query)
    for _ in range(num_pagine):
        s.get_next_results()

    # Controllo se sono stati precedentemente elaborati
    for video in s.videos:
        url = video.watch_url
        if url not in storico_url:
            nuovi_url.append(url)
            storico_url.add(url)  # aggiornamento immediato per evitare duplicati

    # Scrivi i nuovi URL nel file
    with open(file_storico, "a") as f:
        for url in nuovi_url:
            f.write(url + "\n")

    print(f"✅ Trovati {len(nuovi_url)} nuovi video")
    return nuovi_url

def scarica_video_da_youtube(lista_url, cartella_scaricati):
    if not os.path.exists(cartella_scaricati):
        os.makedirs(cartella_scaricati)

    for url in lista_url:
        try:
            yt = YouTube(url)
            stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
            print(f"⬇️ Scaricando: {yt.title}")
            stream.download(output_path=cartella_scaricati)
            print(f"✅ Scaricato: {yt.title}")
        except Exception as e:
            print(f"❌ Errore durante il download di {url}: {e}")

def estrai_immagini_da_video(cartella_video, cartella_output, intervallo_secondi):
    if not os.path.exists(cartella_output):
        os.makedirs(cartella_output)

    for file in os.listdir(cartella_video):
        nome_file, est = os.path.splitext(file)

        percorso_video = os.path.join(cartella_video, file)
        cap = cv2.VideoCapture(percorso_video)

        if not cap.isOpened():
            print(f"Errore nell'aprire il video: {file}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps * intervallo_secondi)

        cartella_immagini_video = os.path.join(cartella_output, nome_file)
        os.makedirs(cartella_immagini_video, exist_ok=True)

        frame_id = 0
        immagine_id = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_id % frame_interval == 0:
                nome_immagine = f"{nome_file}_frame{immagine_id:05d}.jpg"
                percorso_immagine = os.path.join(cartella_immagini_video, nome_immagine)
                cv2.imwrite(percorso_immagine, frame)
                immagine_id += 1

            frame_id += 1

        cap.release()
        print(f"✅ Immagini estratte da '{file}' in '{cartella_immagini_video}'")

#  Configurazioni
query_ricerca = "car accident"
num_pagine = 2  # numero di chiamate a get_next_results()
file_storico_url = "url_scaricati.txt"
cartella_video = "videos"
cartella_output = "frames"
intervallo_secondi = 1

# Cerca nuovi video
nuovi_url = cerca_nuovi_video(query_ricerca, num_pagine, file_storico_url)

#eventuali filtri che si possono aggiungere - non ancora impostati
'''
filters = {
    'upload_date': Filter.get_upload_date('Today'),
    'type': Filter.get_type("Video"),
    'duration': Filter.get_duration("Under 4 minutes"),
    'features': [Filter.get_features("4K"), Filter.get_features("Creative Commons")],
    'sort_by': Filter.get_sort_by("Upload date")
}

s = Search('music', filters=f)
'''

# Scarica nuovi video
scarica_video_da_youtube(nuovi_url, cartella_video)

# Estrai immagini dai nuovi video
estrai_immagini_da_video(cartella_video, cartella_output, intervallo_secondi)


scarica una lista di video impostata manualmente

In [ ]:
import os
import cv2
from pytubefix import YouTube

def filtra_url_nuovi(lista_url, file_storico):
    url_storici = set()
    if os.path.exists(file_storico):
        with open(file_storico, "r") as f:
            url_storici = set(line.strip() for line in f if line.strip())

    nuovi_url = [url for url in lista_url if url not in url_storici]
    return nuovi_url, url_storici

def aggiorna_file_storico(file_storico, nuovi_url):
    with open(file_storico, "a") as f:
        for url in nuovi_url:
            f.write(url + "\n")

def scarica_video_da_youtube(lista_url, cartella_scaricati, file_storico):
    if not os.path.exists(cartella_scaricati):
        os.makedirs(cartella_scaricati)

    nuovi_url, url_storici = filtra_url_nuovi(lista_url, file_storico)

    for url in nuovi_url:
        try:
            yt = YouTube(url)
            stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
            print(f"⬇️ Scaricando: {yt.title}")
            stream.download(output_path=cartella_scaricati)
            print(f"✅ Scaricato: {yt.title}")
        except Exception as e:
            print(f"❌ Errore durante il download di {url}: {e}")
        else:
            aggiorna_file_storico(file_storico, [url])  # aggiorna solo se download riuscito

def estrai_immagini_da_video(cartella_video, cartella_output, intervallo_secondi):
    if not os.path.exists(cartella_output):
        os.makedirs(cartella_output)

    for file in os.listdir(cartella_video):
        nome_file, est = os.path.splitext(file)

        percorso_video = os.path.join(cartella_video, file)
        cap = cv2.VideoCapture(percorso_video)

        if not cap.isOpened():
            print(f"Errore nell'aprire il video: {file}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps * intervallo_secondi)

        cartella_immagini_video = os.path.join(cartella_output, nome_file)
        os.makedirs(cartella_immagini_video, exist_ok=True)

        frame_id = 0
        immagine_id = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_id % frame_interval == 0:
                nome_immagine = f"{nome_file}_frame{immagine_id:05d}.jpg"
                percorso_immagine = os.path.join(cartella_immagini_video, nome_immagine)
                cv2.imwrite(percorso_immagine, frame)
                immagine_id += 1

            frame_id += 1

        cap.release()
        print(f"✅ Immagini estratte da '{file}' in '{cartella_immagini_video}'")

# 🔗 Lista video da scaricare
lista_link_youtube = [
    "https://www.youtube.com/watch?v=1OsLCFd_DCs",
    "https://www.youtube.com/watch?v=mRD0-GxqHVo&pp=0gcJCYQJAYcqIYzv"
]

cartella_video = "videos"
cartella_output = "frames"
file_storico_url = "url_scaricati.txt"

scarica_video_da_youtube(lista_link_youtube, cartella_video, file_storico_url)
estrai_immagini_da_video(cartella_video, cartella_output, intervallo_secondi=2)


download frames

In [ ]:
import shutil
from google.colab import files

output_folder = "/content/frames"

# Path for the zip file that will be created
zip_path = '/content/frames.zip'

# Compress the folder into a .zip file
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', dataset_folder)

# Download the zip file
files.download(zip_path)

In [20]:
!rm -rf /content/frames
!rm -rf /content/videos